# Check out Gios Graph

In [ ]:
edges = pd.read_csv("../../spot2spot.csv")
edges = edges.sort_values('length')
print('len(df): %i' % len(edges))
display(edges.head())
# What are the units of these numbers?
# - length appears to be in km
# - duration appears to be in hours
# What is weight?


old_len = len(edges)
edges = edges.drop_duplicates()
print("old_length: %i, new_length: %i, diff: %i" % (
    old_len, len(edges), old_len-len(edges)))

edges = edges.rename(columns={'origin': 'start', 'destination': 'end'})

# plt.hist(df['length'], bins=100);
# plt.hist(df['duration'], bins=100);
plt.title('Drive Speed Gio\'s Graph')
plt.hist(edges['length']/edges['duration'], bins=100)
plt.ylabel('counts')
plt.xlabel('speed [kmh]')
plt.savefig(os.path.join(plots_path,'gio_drive_speed_hist.png'), dpi=400)

median_speed = (edges['length']/edges['duration']).median()
print('median_speed: %0.2f' % median_speed)


# get nodes
print(len(edges['end'].values), len(edges['start'].values))
print(len(np.unique(edges['end'].values)), len(np.unique(edges['start'].values)))
nodes_raw = np.unique(np.concatenate([edges['start'].values, edges['end'].values]))
nodes = pd.DataFrame(nodes_raw, columns=['id'])
gh_arr = np.array([h3.h3_to_geo(x) for x in nodes_raw])
nodes = nodes.assign(lng = gh_arr[:,0])
nodes = nodes.assign(lat = gh_arr[:,1])


# add lat lng to graph
node_to_lat = dict(zip(nodes.id, nodes.lat))
node_to_lng = dict(zip(nodes.id, nodes.lng))
edges = edges.assign(start_lat = edges['start'].apply(lambda x: node_to_lat[x]))
edges = edges.assign(start_lng = edges['start'].apply(lambda x: node_to_lng[x]))
edges = edges.assign(end_lat = edges['end'].apply(lambda x: node_to_lat[x]))
edges = edges.assign(end_lng = edges['end'].apply(lambda x: node_to_lng[x]))
edges.to_csv("gio_tmp.csv")